In [21]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict,Annotated
import os
from openai import AzureOpenAI 
from dotenv import load_dotenv
from langchain_openai import AzureChatOpenAI  
# InMemorySaver is LangGraph’s in-memory checkpointer implementation.
# It is the simplest checkpointer and is used for storing checkpoints entirely in RAM.
from langgraph.checkpoint.memory import InMemorySaver

In [22]:
load_dotenv()

# ✅ Step 3: Fetch environment variables from the system
azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
azure_api_key = os.getenv("AZURE_OPENAI_API_KEY")
azure_api_version = os.getenv("AZURE_OPENAI_API_VERSION")
azure_deployment = os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME")

client = AzureOpenAI(
    azure_endpoint=azure_endpoint,
    api_key=azure_api_key,
    api_version=azure_api_version
)

# ✅ Use LangChain Azure wrapper — supports structured output
model = AzureChatOpenAI(
    azure_endpoint=azure_endpoint,
    api_key=azure_api_key,
    api_version=azure_api_version,
    azure_deployment=azure_deployment,
    temperature=0,
)

In [23]:
class JokeState(TypedDict):

    topic: str
    joke: str
    explanation: str

In [24]:
def generate_joke(state: JokeState):

    prompt = f'generate a joke on the topic {state["topic"]}'
    response = model.invoke(prompt).content

    return {'joke': response}
def generate_explanation(state: JokeState):

    prompt = f'write an explanation for the joke - {state["joke"]}'
    response = model.invoke(prompt).content

    return {'explanation': response}

In [25]:
graph = StateGraph(JokeState)

graph.add_node('generate_joke', generate_joke)
graph.add_node('generate_explanation', generate_explanation)

graph.add_edge(START, 'generate_joke')
graph.add_edge('generate_joke', 'generate_explanation')
graph.add_edge('generate_explanation', END)

checkpointer = InMemorySaver()

In [26]:
workflow = graph.compile(checkpointer=checkpointer)

config1 = {"configurable": {"thread_id": "1"}}
workflow.invoke({'topic':'pizza'}, config=config1)

{'topic': 'pizza',
 'joke': 'Why did the pizza maker go broke?\n\nBecause he just couldn’t make enough dough! 🍕',
 'explanation': 'This joke is a play on words, or a pun, that uses the double meaning of the word "dough." In the context of pizza making, "dough" refers to the mixture of flour, water, and yeast used to make pizza crust. However, "dough" is also a slang term for money. \n\nThe joke suggests that the pizza maker went broke because he couldn\'t "make enough dough." On one level, it humorously implies he couldn\'t make enough pizza dough to keep his business running. On another level, it means he wasn\'t earning enough money to stay financially stable. The humor comes from the clever use of the word\'s dual meanings. 🍕'}

In [27]:
workflow.get_state(config1)

StateSnapshot(values={'topic': 'pizza', 'joke': 'Why did the pizza maker go broke?\n\nBecause he just couldn’t make enough dough! 🍕', 'explanation': 'This joke is a play on words, or a pun, that uses the double meaning of the word "dough." In the context of pizza making, "dough" refers to the mixture of flour, water, and yeast used to make pizza crust. However, "dough" is also a slang term for money. \n\nThe joke suggests that the pizza maker went broke because he couldn\'t "make enough dough." On one level, it humorously implies he couldn\'t make enough pizza dough to keep his business running. On another level, it means he wasn\'t earning enough money to stay financially stable. The humor comes from the clever use of the word\'s dual meanings. 🍕'}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0c121d-da29-678d-8002-3b680f3dd150'}}, metadata={'source': 'loop', 'step': 2, 'parents': {}}, created_at='2025-11-14T06:19:24.498716+00:00', paren

In [28]:
list(workflow.get_state_history(config1))

[StateSnapshot(values={'topic': 'pizza', 'joke': 'Why did the pizza maker go broke?\n\nBecause he just couldn’t make enough dough! 🍕', 'explanation': 'This joke is a play on words, or a pun, that uses the double meaning of the word "dough." In the context of pizza making, "dough" refers to the mixture of flour, water, and yeast used to make pizza crust. However, "dough" is also a slang term for money. \n\nThe joke suggests that the pizza maker went broke because he couldn\'t "make enough dough." On one level, it humorously implies he couldn\'t make enough pizza dough to keep his business running. On another level, it means he wasn\'t earning enough money to stay financially stable. The humor comes from the clever use of the word\'s dual meanings. 🍕'}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0c121d-da29-678d-8002-3b680f3dd150'}}, metadata={'source': 'loop', 'step': 2, 'parents': {}}, created_at='2025-11-14T06:19:24.498716+00:00', pare

In [29]:
config2 = {"configurable": {"thread_id": "2"}}
workflow.invoke({'topic':'love'}, config=config2)

{'topic': 'love',
 'joke': 'Why did the magnet break up with the paperclip?\n\nIt felt like the attraction was one-sided!',
 'explanation': 'This joke plays on the double meaning of "attraction." In the context of magnets and paperclips, "attraction" refers to the physical force that pulls the two objects together. However, in relationships, "attraction" refers to emotional or romantic interest between two people. \n\nThe punchline, "It felt like the attraction was one-sided," suggests that the magnet felt it was doing all the work in the relationship, as magnets are the ones that actively attract paperclips, while paperclips are passive and don\'t generate their own magnetic pull. This mirrors the idea of an unbalanced relationship where only one person feels invested, making the joke both clever and relatable!'}

In [30]:
list(workflow.get_state_history(config2))

[StateSnapshot(values={'topic': 'love', 'joke': 'Why did the magnet break up with the paperclip?\n\nIt felt like the attraction was one-sided!', 'explanation': 'This joke plays on the double meaning of "attraction." In the context of magnets and paperclips, "attraction" refers to the physical force that pulls the two objects together. However, in relationships, "attraction" refers to emotional or romantic interest between two people. \n\nThe punchline, "It felt like the attraction was one-sided," suggests that the magnet felt it was doing all the work in the relationship, as magnets are the ones that actively attract paperclips, while paperclips are passive and don\'t generate their own magnetic pull. This mirrors the idea of an unbalanced relationship where only one person feels invested, making the joke both clever and relatable!'}, next=(), config={'configurable': {'thread_id': '2', 'checkpoint_ns': '', 'checkpoint_id': '1f0c121e-054a-6ff2-8002-62c3150c53c3'}}, metadata={'source': '

# Time Travel


In [36]:
workflow.get_state({"configurable": {"thread_id": "2", "checkpoint_id": "1f0c121d-da72-61fb-8000-fbc7661cffdc"}})

StateSnapshot(values={'topic': 'love'}, next=('generate_joke',), config={'configurable': {'thread_id': '2', 'checkpoint_id': '1f0c121d-da72-61fb-8000-fbc7661cffdc'}}, metadata={'source': 'loop', 'step': 0, 'parents': {}}, created_at='2025-11-14T06:19:24.528470+00:00', parent_config={'configurable': {'thread_id': '2', 'checkpoint_ns': '', 'checkpoint_id': '1f0c121d-da6f-6d82-bfff-6ada0643f458'}}, tasks=(PregelTask(id='2b105995-d3c7-10dc-0f05-5a2db27d8799', name='generate_joke', path=('__pregel_pull', 'generate_joke'), error=None, interrupts=(), state=None, result={'joke': 'Why did the magnet break up with the paperclip?\n\nIt felt like the attraction was one-sided!'}),), interrupts=())

In [37]:
workflow.invoke(None, {"configurable": {"thread_id": "2", "checkpoint_id": "1f0c121d-da72-61fb-8000-fbc7661cffdc"}})

{'topic': 'love',
 'joke': "Sure! Here's a love-themed joke for you:\n\nWhy did the magnet break up with the refrigerator?\n\nIt felt like it was being too clingy!",
 'explanation': 'This joke plays on the literal and figurative meanings of "clingy." Magnets physically cling to refrigerators because of their magnetic attraction, which is a natural property of magnets. In relationships, "clingy" is a term used to describe someone who is overly attached or dependent on their partner, often to an excessive or uncomfortable degree. \n\nThe humor comes from imagining the magnet and refrigerator as a couple, with the magnet deciding to "break up" because it feels the refrigerator is being "too clingy." The twist is that the magnet itself is the one doing the clinging, making the joke a clever play on words and situational irony.'}